# DMTA Vitrimer Analysis: Temperature-Dependent Bond Exchange

Vitrimers are a class of covalent adaptable networks (CANs) that undergo reversible bond exchange reactions (BER) with Arrhenius temperature dependence. DMTA temperature sweeps capture the topology freezing transition T_v where BER kinetics freeze out.

## Learning Objectives
- Understand vitrimer topology freezing and BER kinetics
- Generate and analyze temperature-sweep DMTA data
- Extract activation energy E_a from Arrhenius behavior
- Fit HVM (Hybrid Vitrimer Model) to isothermal SAOS data
- Interpret T_v from tan(δ) peak

**Estimated Time:** 12 minutes

In [1]:
import gc
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np

if os.path.abspath(os.path.join(os.getcwd(), '../..')) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

FAST_MODE = os.environ.get('FAST_MODE', '1') == '1'
print(f'FAST_MODE: {FAST_MODE}')

FAST_MODE: True


## 1. The Physics: Vitrimer Topology Freezing

Vitrimers contain three types of crosslinks:
- **Permanent** (P): Covalent, static (cannot exchange)
- **Exchangeable** (E): Covalent, dynamic via BER: k_BER = ν₀·exp(-E_a/RT)
- **Dissociative** (D): Physical bonds (optional)

At high T: BER is fast → stress relaxation → E' drops with decreasing frequency
At low T (below T_v): BER freezes → permanent network → E' plateau

The BER rate follows Arrhenius kinetics:
$$k_{BER}(T) = \nu_0 \cdot \exp\left(-\frac{E_a}{RT}\right)$$

The effective relaxation time of the exchangeable network is:
$$\tau_E = \frac{1}{2 k_{BER}}$$

The factor of 2 arises because both the stress tensor and natural-state tensor relax toward each other.

## 2. Synthetic DMTA Temperature Sweep

We generate E'(T) and E''(T) at a fixed frequency (ω = 1 rad/s) across a temperature range. This simulates a DMTA temperature sweep experiment.

In [2]:
# Temperature range (K)
T_range = np.linspace(300, 500, 30 if FAST_MODE else 60)  # 27°C to 227°C
omega_fixed = 1.0  # 1 rad/s - typical DMTA frequency

# HVM-like parameters (G-space)
G_P = 5e3     # Permanent network modulus (Pa)
G_E = 3e3     # Exchangeable network modulus (Pa)
nu_0 = 1e10   # Attempt frequency (1/s)
E_a = 80e3    # Activation energy (J/mol)
R = 8.314     # Gas constant (J/mol/K)

# Poisson's ratio for conversion to E*
nu = 0.5
factor = 2 * (1 + nu)  # = 3.0 for rubber-like vitrimer

# Compute effective relaxation time at each temperature
k_BER = nu_0 * np.exp(-E_a / (R * T_range))
tau_E = 1 / (2 * k_BER)  # factor of 2 for evolving natural state

# Maxwell mode contribution from exchangeable network
omega_tau = omega_fixed * tau_E
G_E_prime = G_E * omega_tau**2 / (1 + omega_tau**2)
G_E_double_prime = G_E * omega_tau / (1 + omega_tau**2)

# Total moduli (G_P is elastic, always contributes to storage)
G_prime_total = G_P + G_E_prime
G_double_prime_total = G_E_double_prime

# Convert to E* (tension)
E_prime_total = factor * G_prime_total
E_double_prime_total = factor * G_double_prime_total

# Add 2% noise
noise_p = 1 + 0.02 * np.random.randn(len(T_range))
noise_pp = 1 + 0.02 * np.random.randn(len(T_range))
E_prime_noisy = E_prime_total * noise_p
E_double_prime_noisy = E_double_prime_total * np.abs(noise_pp)

print(f'Temperature range: {T_range[0]:.0f} - {T_range[-1]:.0f} K')
print(f'                   {T_range[0]-273.15:.0f} - {T_range[-1]-273.15:.0f} °C')
print(f'Fixed frequency: {omega_fixed} rad/s')
print(f'\nMaterial parameters:')
print(f'  G_P = {G_P:.0e} Pa (permanent network)')
print(f'  G_E = {G_E:.0e} Pa (exchangeable network)')
print(f'  ν₀  = {nu_0:.0e} 1/s (attempt frequency)')
print(f'  E_a = {E_a/1e3:.0f} kJ/mol (activation energy)')
print(f'\nConversion factor (ν={nu}): {factor:.1f}')

Temperature range: 300 - 500 K
                   27 - 227 °C
Fixed frequency: 1.0 rad/s

Material parameters:
  G_P = 5e+03 Pa (permanent network)
  G_E = 3e+03 Pa (exchangeable network)
  ν₀  = 1e+10 1/s (attempt frequency)
  E_a = 80 kJ/mol (activation energy)

Conversion factor (ν=0.5): 3.0


## 3. Temperature Sweep Visualization

The characteristic vitrimer signature:
- **E'(T)**: Plateau at low T (frozen BER) → decrease at high T (active BER)
- **E''(T)**: Peak at the transition temperature
- **tan(δ)(T)**: Peak defines topology freezing temperature T_v

In [3]:
T_C = T_range - 273.15  # Convert to Celsius for plotting
tan_delta = E_double_prime_noisy / E_prime_noisy

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# E' vs T
ax = axes[0]
ax.plot(T_C, E_prime_noisy, 'ro-', lw=2, ms=4, label="E'")
ax.axhline(factor * G_P, color='gray', ls='--', alpha=0.5, label=f'E_P = {factor*G_P:.0e} Pa')
ax.set_xlabel('Temperature (°C)')
ax.set_ylabel("E' (Pa)")
ax.set_title("Storage Modulus E'(T)")
ax.legend()
ax.grid(alpha=0.3)

# E'' vs T
ax = axes[1]
ax.plot(T_C, E_double_prime_noisy, 'bs-', lw=2, ms=4, label='E"')
ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('E" (Pa)')
ax.set_title('Loss Modulus E"(T)')
ax.legend()
ax.grid(alpha=0.3)

# tan(δ) vs T
ax = axes[2]
ax.plot(T_C, tan_delta, 'g^-', lw=2, ms=4, label='tan(δ)')
T_v_idx = np.argmax(tan_delta)
T_v = T_range[T_v_idx]
ax.axvline(T_v - 273.15, color='red', ls='--', alpha=0.7, label=f'T_v ≈ {T_v-273.15:.0f}°C')
ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('tan(δ)')
ax.set_title('Loss Tangent tan(δ)(T)')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.close('all')

print(f'\nTopology freezing temperature T_v = {T_v:.0f} K ({T_v - 273.15:.0f} °C)')
print(f'tan(δ)_max = {tan_delta[T_v_idx]:.4f} at T_v')


Topology freezing temperature T_v = 410 K (137 °C)
tan(δ)_max = 0.2397 at T_v


## 4. Topology Freezing Temperature Extraction

The T_v is commonly defined as the temperature at which tan(δ) peaks. At this point, the BER rate k_BER(T_v) ≈ ω, meaning the relaxation time τ_E(T_v) ≈ 1/ω.

In [4]:
# Extract T_v from tan(δ) peak
T_v_idx = np.argmax(tan_delta)
T_v = T_range[T_v_idx]
tau_at_Tv = tau_E[T_v_idx]

print(f'Topology freezing transition:')
print(f'  T_v = {T_v:.2f} K ({T_v - 273.15:.1f} °C)')
print(f'  τ_E at T_v = {tau_at_Tv:.4f} s')
print(f'  ω·τ_E at T_v = {omega_fixed * tau_at_Tv:.4f} (should be ~1 for tan(δ) peak)')
print(f'\nPhysical interpretation:')
print(f'  T < {T_v-273.15:.0f}°C: BER frozen → vitrimer behaves as permanent elastomer')
print(f'  T > {T_v-273.15:.0f}°C: BER active → vitrimer can flow and be reshaped')

Topology freezing transition:
  T_v = 410.34 K (137.2 °C)
  τ_E at T_v = 0.7637 s
  ω·τ_E at T_v = 0.7637 (should be ~1 for tan(δ) peak)

Physical interpretation:
  T < 137°C: BER frozen → vitrimer behaves as permanent elastomer
  T > 137°C: BER active → vitrimer can flow and be reshaped


## 5. Arrhenius Analysis: Activation Energy Extraction

From the Arrhenius equation:
$$\ln(k_{BER}) = \ln(\nu_0) - \frac{E_a}{R} \cdot \frac{1}{T}$$

A plot of ln(k_BER) vs 1/T gives a straight line with slope -E_a/R.

In [5]:
# Arrhenius plot: ln(1/tau_E) vs 1/T
# Since tau_E = 1/(2*k_BER), we have k_BER = 1/(2*tau_E)
one_over_T = 1 / T_range
ln_k_BER = np.log(1 / (2 * tau_E))

# Linear fit to extract E_a
coeffs = np.polyfit(one_over_T, ln_k_BER, deg=1)
slope = coeffs[0]
intercept = coeffs[1]
E_a_extracted = -slope * R
nu_0_extracted = np.exp(intercept)

# Predicted line
ln_k_fit = slope * one_over_T + intercept

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(one_over_T * 1000, ln_k_BER, 'ko', ms=5, label='Data')
ax.plot(one_over_T * 1000, ln_k_fit, 'r-', lw=2, label=f'Linear fit: E_a = {E_a_extracted/1e3:.1f} kJ/mol')
ax.set_xlabel('1000/T (1/K)')
ax.set_ylabel('ln(k_BER) [ln(1/s)]')
ax.set_title('Arrhenius Plot: BER Rate vs Inverse Temperature')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.close('all')

print(f'Arrhenius fit results:')
print(f'  True E_a   = {E_a/1e3:.1f} kJ/mol')
print(f'  Fitted E_a = {E_a_extracted/1e3:.1f} kJ/mol')
print(f'  Error      = {abs(E_a_extracted - E_a)/E_a * 100:.2f}%')
print(f'\n  True ν₀   = {nu_0:.2e} 1/s')
print(f'  Fitted ν₀ = {nu_0_extracted:.2e} 1/s')
print(f'  Error     = {abs(nu_0_extracted - nu_0)/nu_0 * 100:.2f}%')

Arrhenius fit results:
  True E_a   = 80.0 kJ/mol
  Fitted E_a = 80.0 kJ/mol
  Error      = 0.00%

  True ν₀   = 1.00e+10 1/s
  Fitted ν₀ = 1.00e+10 1/s
  Error     = 0.00%


## 6. HVM Isothermal SAOS Fit

To demonstrate the HVM model with DMTA-style data, we fit isothermal SAOS data at T = 400 K (above T_v). The HVM includes built-in Arrhenius kinetics for the exchangeable network.

In [6]:
from rheojax.models.hvm import HVMLocal

# Generate isothermal SAOS data at T = 400K (above T_v)
T_fit = 400.0  # K
omega_saos = np.logspace(-2, 2, 40 if FAST_MODE else 80)

# True parameters at this temperature
k_fit = nu_0 * np.exp(-E_a / (R * T_fit))
tau_fit = 1 / (2 * k_fit)
omega_tau_fit = omega_saos * tau_fit

# G*(ω) for Maxwell mode (permanent + exchangeable)
G_prime_fit = G_P + G_E * omega_tau_fit**2 / (1 + omega_tau_fit**2)
G_double_prime_fit = G_E * omega_tau_fit / (1 + omega_tau_fit**2)
G_star_fit = G_prime_fit + 1j * G_double_prime_fit

# Convert to E* (tension)
E_star_fit = factor * G_star_fit

# Add 1% noise
noise_r = 1 + 0.01 * np.random.randn(len(omega_saos))
noise_i = 1 + 0.01 * np.random.randn(len(omega_saos))
E_star_fit_noisy = np.real(E_star_fit) * noise_r + 1j * np.imag(E_star_fit) * np.abs(noise_i)

print(f'Isothermal SAOS at T = {T_fit:.0f} K ({T_fit - 273.15:.0f} °C)')
print(f'  k_BER(T) = {k_fit:.4e} 1/s')
print(f'  τ_E(T)   = {tau_fit:.4e} s')
print(f'  Frequency range: {omega_saos.min():.2e} - {omega_saos.max():.2e} rad/s')

Isothermal SAOS at T = 400 K (127 °C)
  k_BER(T) = 3.5702e-01 1/s
  τ_E(T)   = 1.4005e+00 s
  Frequency range: 1.00e-02 - 1.00e+02 rad/s


In [7]:
# Create HVM model using partial_vitrimer factory method
# Start with reasonable initial guesses
model = HVMLocal.partial_vitrimer(G_P=1e4, G_E=1e4, nu_0=1e8, E_a=60e3)

# Set temperature parameter
model.parameters.set_value('T', T_fit)

# Fit to E* data
model.fit(
    omega_saos, E_star_fit_noisy,
    test_mode='oscillation',
    deformation_mode='tension',
    poisson_ratio=0.5,
)

# Extract fitted parameters (G-space)
G_P_fit = model.parameters.get_value('G_P')
G_E_fit = model.parameters.get_value('G_E')
nu_0_fit = model.parameters.get_value('nu_0')
E_a_fit = model.parameters.get_value('E_a')

print('\nHVM Fitted Parameters (G-space):')
print(f'  G_P (permanent)   = {G_P_fit:.4e} Pa  (true: {G_P:.4e})')
print(f'  G_E (exchangeable)= {G_E_fit:.4e} Pa  (true: {G_E:.4e})')
print(f'  ν₀ (attempt freq) = {nu_0_fit:.4e} 1/s (true: {nu_0:.4e})')
print(f'  E_a (activation)  = {E_a_fit/1e3:.1f} kJ/mol (true: {E_a/1e3:.1f})')

print('\nParameter errors:')
print(f'  G_P error  = {abs(G_P_fit - G_P)/G_P * 100:.2f}%')
print(f'  G_E error  = {abs(G_E_fit - G_E)/G_E * 100:.2f}%')
print(f'  ν₀ error   = {abs(nu_0_fit - nu_0)/nu_0 * 100:.2f}%')
print(f'  E_a error  = {abs(E_a_fit - E_a)/E_a * 100:.2f}%')

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=6 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=1.607662s


INFO:nlsq.least_squares:Convergence reason=Both `ftol` and `xtol` termination conditions are satisfied. | iterations=19 | final_cost=0.0016 | elapsed=1.608s | final_gradient_norm=2.2076



HVM Fitted Parameters (G-space):
  G_P (permanent)   = 5.0048e+03 Pa  (true: 5.0000e+03)
  G_E (exchangeable)= 2.9934e+03 Pa  (true: 3.0000e+03)
  ν₀ (attempt freq) = 2.5178e+08 1/s (true: 1.0000e+10)
  E_a (activation)  = 63.7 kJ/mol (true: 80.0)

Parameter errors:
  G_P error  = 0.10%
  G_E error  = 0.22%
  ν₀ error   = 97.48%
  E_a error  = 20.32%


In [8]:
# Predict and compare
E_pred = model.predict(omega_saos, test_mode='oscillation')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# E' and E'' comparison
ax = axes[0]
ax.loglog(omega_saos, np.real(E_star_fit_noisy), 'ro', ms=4, alpha=0.5, label="E' data")
ax.loglog(omega_saos, np.imag(E_star_fit_noisy), 'bs', ms=4, alpha=0.5, label='E" data')
ax.loglog(omega_saos, np.real(E_pred), 'r-', lw=2, label="E' HVM fit")
ax.loglog(omega_saos, np.imag(E_pred), 'b-', lw=2, label='E" HVM fit')
ax.set_xlabel('ω (rad/s)')
ax.set_ylabel('Modulus (Pa)')
ax.set_title(f'HVM Fit to Isothermal SAOS at {T_fit-273.15:.0f}°C')
ax.legend()
ax.grid(alpha=0.3)

# tan(δ) comparison
ax = axes[1]
tan_d_data = np.imag(E_star_fit_noisy) / np.real(E_star_fit_noisy)
tan_d_fit = np.imag(E_pred) / np.real(E_pred)
ax.semilogx(omega_saos, tan_d_data, 'go', ms=4, alpha=0.5, label='Data')
ax.semilogx(omega_saos, tan_d_fit, 'g-', lw=2, label='HVM fit')
ax.set_xlabel('ω (rad/s)')
ax.set_ylabel('tan(δ)')
ax.set_title('Loss Tangent tan(δ)')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.close('all')

# Compute R²
residual = np.abs(E_star_fit_noisy) - np.abs(E_pred)
ss_res = np.sum(residual**2)
ss_tot = np.sum((np.abs(E_star_fit_noisy) - np.mean(np.abs(E_star_fit_noisy)))**2)
R2 = 1 - ss_res / ss_tot
print(f'\nFit quality: R² = {R2:.6f}')


Fit quality: R² = 0.995648


## Key Takeaways

- **Vitrimer DMTA signature**: E'(T) plateau → drop, tan(δ)(T) peak at T_v
- **Topology freezing T_v**: Defined by tan(δ) maximum, where k_BER ≈ ω
- **Arrhenius kinetics**: k_BER = ν₀·exp(-E_a/RT) controls temperature dependence
- **E_a extraction**: Linear Arrhenius plot of ln(k_BER) vs 1/T
- **HVM model**: Built-in Arrhenius kinetics, factor-of-2 in τ_E for natural-state evolution
- **deformation_mode='tension'**: Enables direct DMTA (E*) fitting with automatic G*↔E* conversion

## Physical Insight

The activation energy E_a controls the **steepness** of the T_v transition:
- **Low E_a** (~40 kJ/mol): Broad transition, gradual modulus drop
- **High E_a** (~150 kJ/mol): Sharp transition, narrow tan(δ) peak

Typical vitrimer E_a values range from 60-120 kJ/mol, depending on the exchange mechanism (transesterification, transamination, etc.).

## Next Steps

- `06_dmta_model_selection.ipynb`: Systematic model comparison for DMTA data
- See HVM tutorial notebooks in `examples/hvm/` for startup, relaxation, and LAOS protocols

In [9]:
del model
gc.collect()
jax.clear_caches()
print('Cleanup complete')

Cleanup complete
